# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [195]:
# jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [196]:
# The path to our CSV file
file = "../../PartI-WeatherPy/starter_code/output.csv"

# Read our Kickstarter data into pandas
cities_df = pd.read_csv(file)

cities_df.head()

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tuatapere,NZ,56,1580083741,60,-46.13,167.68,68.00,8.99
1,Taman,RU,40,1580083741,86,45.21,36.72,41.00,8.95
2,Yellowknife,CA,75,1580083741,84,62.46,-114.35,3.20,8.05
3,Hilo,US,75,1580083741,58,19.73,-155.09,82.40,12.75
4,Cape Town,ZA,75,1580083591,77,-33.93,18.42,62.01,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [197]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store humidity as the weight and convert to float
weight = cities_df["Humidity"].astype(float)


In [198]:
# Setting the zoom level and map center explicitly:
new_york_coordinates = (40.75, -74.00)

# Plot Heatmap
fig = gmaps.figure(center=new_york_coordinates, zoom_level=3)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# make the heatmap transparent
heat_layer.opacity = 0.4

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [199]:
# Setting the zoom level and map center explicitly:
new_york_coordinates = (40.75, -74.00)

# Plot Heatmap as a Hybrid map
fig = gmaps.figure(center=new_york_coordinates, zoom_level=2, map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# make the heatmap transparent
heat_layer.opacity = 0.4 

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [200]:
cities_df = cities_df.dropna()
cities_df.count()

City          547
Country       547
Cloudiness    547
Date          547
Humidity      547
Lat           547
Lng           547
Max Temp      547
Wind Speed    547
dtype: int64

In [201]:
# A max temperature lower than 80 degrees but higher than 70
vacation_destinations_temp_df = cities_df[(cities_df['Max Temp'] < 80.00) & (cities_df['Max Temp'] > 70.00)]
print(vacation_destinations_temp_df.count())
vacation_destinations_temp_df.head()

City          122
Country       122
Cloudiness    122
Date          122
Humidity      122
Lat           122
Lng           122
Max Temp      122
Wind Speed    122
dtype: int64


,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
8,Lucea,JM,20,1580083742,69,18.45,-78.17,78.80,9.17
10,Saint-Joseph,RE,75,1580083743,78,-21.37,55.62,73.40,8.05
12,Road Town,VG,40,1580083743,94,18.42,-64.62,78.80,1.12
16,Bengkulu,ID,85,1580083744,86,-3.80,102.27,74.95,2.77
17,Kintampo,GH,53,1580083744,10,8.06,-1.73,75.65,1.52


In [202]:
# Wind speed less than 10 mph
vacation_destinations_temp_wind_df = vacation_destinations_temp_df[vacation_destinations_temp_df['Wind Speed'] <= 10.00]
vacation_destinations_temp_wind_df.count()
vacation_destinations_temp_wind_df.head()

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
8,Lucea,JM,20,1580083742,69,18.45,-78.17,78.80,9.17
10,Saint-Joseph,RE,75,1580083743,78,-21.37,55.62,73.40,8.05
12,Road Town,VG,40,1580083743,94,18.42,-64.62,78.80,1.12
16,Bengkulu,ID,85,1580083744,86,-3.80,102.27,74.95,2.77
17,Kintampo,GH,53,1580083744,10,8.06,-1.73,75.65,1.52


In [212]:
# Zero cloudiness
vacation_destinations_temp_wind_cloud_df = vacation_destinations_temp_wind_df[vacation_destinations_temp_wind_df['Cloudiness'] == 0.00]
print(vacation_destinations_temp_wind_cloud_df.count())
vacation_destinations_temp_wind_cloud_df =vacation_destinations_temp_wind_cloud_df.reset_index(drop=True)
vacation_destinations_temp_wind_cloud_df

City          5
Country       5
Cloudiness    5
Date          5
Humidity      5
Lat           5
Lng           5
Max Temp      5
Wind Speed    5
dtype: int64


,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Busselton,AU,0,1580083749,52,-33.65,115.33,72.00,3.00
1,Bunbury,AU,0,1580083753,52,-33.33,115.63,72.00,3.00
2,Mangrol,IN,0,1580083768,67,21.12,70.12,70.52,9.13
3,Tomatlan,MX,0,1580083814,69,19.93,-105.25,74.62,7.00
4,Djambala,CG,0,1580083840,78,-2.54,14.75,71.71,5.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [213]:
# Store into variable named hotel_df
hotel_df = vacation_destinations_temp_wind_cloud_df
hotel_df

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Busselton,AU,0,1580083749,52,-33.65,115.33,72.00,3.00
1,Bunbury,AU,0,1580083753,52,-33.33,115.63,72.00,3.00
2,Mangrol,IN,0,1580083768,67,21.12,70.12,70.52,9.13
3,Tomatlan,MX,0,1580083814,69,19.93,-105.25,74.62,7.00
4,Djambala,CG,0,1580083840,78,-2.54,14.75,71.71,5.19


In [214]:
# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ''
hotel_df['Hotel Rating'] = ''
hotel_df

,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Rating
0,Busselton,AU,0,1580083749,52,-33.65,115.33,72.00,3.00,,
1,Bunbury,AU,0,1580083753,52,-33.33,115.63,72.00,3.00,,
2,Mangrol,IN,0,1580083768,67,21.12,70.12,70.52,9.13,,
3,Tomatlan,MX,0,1580083814,69,19.93,-105.25,74.62,7.00,,
4,Djambala,CG,0,1580083840,78,-2.54,14.75,71.71,5.19,,


In [235]:
# Set parameters to search for hotels with 5000 meters
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}



# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    best_rating = 0.0
    count = 0
    hotel_list = []
    name = ''
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request 
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    print(json.dumps(name_address['results'], indent=4, sort_keys=True))
    
    
    try:
#         print(f"Closest hotel is {name_address['results'][0]['name']} with rating {name_address['results'][0]['rating']}.")
        for item in name_address['results']:
            
            count +=1
                         
            # print(item['name'], item['rating'])
            hotel = (item['name'], item['rating']) 
            hotel_list.append(hotel)
        print(count)
        print(hotel_list)  
              
        # Searching for the hotel with the best rating          
        for item in hotel_list:
            if best_rating < item[1]:
                best_rating = item[1]
                name = item[0]
                print(name, best_rating)
                    
              
        hotel_df.loc[index, 'Hotel Name'] = name
        hotel_df.loc[index, 'Hotel Rating'] = best_rating
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------------------")
    

[
    {
        "geometry": {
            "location": {
                "lat": -33.6583819,
                "lng": 115.2742265
            },
            "viewport": {
                "northeast": {
                    "lat": -33.65703527010728,
                    "lng": 115.2757757298927
                },
                "southwest": {
                    "lat": -33.65973492989272,
                    "lng": 115.2730760701073
                }
            }
        },
        "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
        "id": "5c2b7738ab307ccd2a3ac5262e06122f71e9d861",
        "name": "Abbey Beach Resort",
        "opening_hours": {
            "open_now": true
        },
        "photos": [
            {
                "height": 2250,
                "html_attributions": [
                    "<a href=\"https://maps.google.com/maps/contrib/106291660568444562147\">A Google User</a>"
                ],
                "photo_reference": "CmRaAA

[
    {
        "geometry": {
            "location": {
                "lat": -33.3447153,
                "lng": 115.6230951
            },
            "viewport": {
                "northeast": {
                    "lat": -33.34333062010728,
                    "lng": 115.6242832298928
                },
                "southwest": {
                    "lat": -33.34603027989272,
                    "lng": 115.6215835701073
                }
            }
        },
        "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
        "id": "414081828849849bda637fc2a02727f97ad2201b",
        "name": "Bunbury Seaview Apartments",
        "opening_hours": {
            "open_now": true
        },
        "photos": [
            {
                "height": 787,
                "html_attributions": [
                    "<a href=\"https://maps.google.com/maps/contrib/105191208661562570898\">Bunbury Seaview Apartments</a>"
                ],
                "photo

[
    {
        "geometry": {
            "location": {
                "lat": 21.1222526,
                "lng": 70.1202768
            },
            "viewport": {
                "northeast": {
                    "lat": 21.12369047989272,
                    "lng": 70.12165037989271
                },
                "southwest": {
                    "lat": 21.12099082010728,
                    "lng": 70.11895072010726
                }
            }
        },
        "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png",
        "id": "c050dac2a95192a61a95419c8a1565510f82353f",
        "name": "Kaveri Hotel",
        "opening_hours": {
            "open_now": false
        },
        "photos": [
            {
                "height": 2304,
                "html_attributions": [
                    "<a href=\"https://maps.google.com/maps/contrib/111215600056463914028\">Belim Sajid Khan</a>"
                ],
                "photo_reference": "CmRaAAAA5

[
    {
        "geometry": {
            "location": {
                "lat": -2.5401049,
                "lng": 14.7475519
            },
            "viewport": {
                "northeast": {
                    "lat": -2.538784420107278,
                    "lng": 14.74889057989273
                },
                "southwest": {
                    "lat": -2.541484079892723,
                    "lng": 14.74619092010728
                }
            }
        },
        "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
        "id": "d1d9c7fc60aca0986455ab339b54299c535afcfd",
        "name": "Hotel Ngangouoni Ipari Salem",
        "place_id": "ChIJJRwBNVIXbhoRtorCJOICA5Q",
        "plus_code": {
            "compound_code": "FP5X+X2 Djambala, Congo - Brazzaville",
            "global_code": "6F9PFP5X+X2"
        },
        "rating": 4,
        "reference": "ChIJJRwBNVIXbhoRtorCJOICA5Q",
        "scope": "GOOGLE",
        "types": [
            "lodging"

In [216]:
hotel_df


,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Rating
0,Busselton,AU,0,1580083749,52,-33.65,115.33,72.00,3.00,Fairlawn Estate,5
1,Bunbury,AU,0,1580083753,52,-33.33,115.63,72.00,3.00,Boathouse Bunbury Bed and Breakfast,5
2,Mangrol,IN,0,1580083768,67,21.12,70.12,70.52,9.13,Kaveri Hotel,3.9
3,Tomatlan,MX,0,1580083814,69,19.93,-105.25,74.62,7.00,Hotel Misión San Miguel,4.2
4,Djambala,CG,0,1580083840,78,-2.54,14.75,71.71,5.19,Hotel Vintsie,4.3


In [245]:
# Plot markers on top of the heatmap.
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store humidity as the weight and convert to float
weight = cities_df["Humidity"].astype(float)

# Setting the zoom level and map center explicitly:
hotel_coordinates = (hotel_df.iloc[0, 5], hotel_df.iloc[0, 6])

# Plot Heatmap
fig = gmaps.figure(center=hotel_coordinates, zoom_level=1)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)

# make the heatmap transparent
heat_layer.opacity = 0.5


# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>

<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display Map
# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))